In [65]:
import pandas as pd
import numpy as np

<h1>Rotten movies</h1>

In [66]:
from datasets import load_dataset

rotten_data = load_dataset("rotten_tomatoes")

Using custom data configuration default
Reusing dataset rotten_tomatoes_movie_review (/home/diego/.cache/huggingface/datasets/rotten_tomatoes_movie_review/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46)


  0%|          | 0/3 [00:00<?, ?it/s]

In [67]:
print(f"Total length of polarity rotten test {len(rotten_data['test'])}")
print(f"Total length of polarity rotten train {len(rotten_data['train'])}")

Total length of polarity rotten test 1066
Total length of polarity rotten train 8530


We merge train and test partitions

In [68]:
texts = rotten_data["test"]["text"]
labels = rotten_data["test"]["label"]
d = {"text": texts,
     "labels": labels,
}
df_test = pd.DataFrame(data=d)

texts = rotten_data["train"]["text"]
labels = rotten_data["train"]["label"]
d = {"text": texts,
     "labels": labels,
     }
df_train = pd.DataFrame(data=d)

df_rotten = pd.concat([df_train, df_test])
df_rotten["sentiment"] = df_rotten.apply(lambda row: "positive" if row['labels'] else "negative", axis=1)
print(f"df len:{len(df_rotten)}")
df_rotten.head(2)

df len:9596


,text,labels,sentiment
0,the rock is destined to be the 21st century's ...,1,positive
1,"the gorgeously elaborate continuation of "" the...",1,positive


In [69]:
texts = df_rotten["text"].values
lens = [len(el.split(" ")) for el in texts]
print(f"Max len: {np.max(lens)}")
print(f"Mean len: {np.mean(lens)}")
print(f"Min len: {np.min(lens)}")

df_rotten["sentiment"].value_counts()

Max len: 59
Mean len: 21.017507294706128
Min len: 1


positive    4798
negative    4798
Name: sentiment, dtype: int64

In [77]:
# filter out reviews longer than 2000 and shorter than 10 words
texts = df_rotten["text"].values
df_rotten["review_len"] = [len(el.split(" ")) for el in texts]
df_rotten = df_rotten[(df_rotten["review_len"] <= 2000) & (df_rotten["review_len"] >= 9)].copy()

lens = [len(el) for el in df_rotten["text"].values]
print(f"Max len: {np.max(df_rotten['review_len'].values)}")
print(f"Mean len: {np.mean(df_rotten['review_len'].values)}")
print(f"Min len: {np.min(df_rotten['review_len'].values)}")

print(f"df len:{len(df_rotten)}")
df_rotten["labels"].value_counts()

Max len: 59
Mean len: 22.464150943396227
Min len: 9
df len:8745


1    4389
0    4356
Name: labels, dtype: int64

In [71]:
df_pos = df_rotten[df_rotten["labels"] == 1].copy()
df_neg = df_rotten[df_rotten["labels"] == 0].copy()

## Sample train-test split 7.5k-1k

In [72]:
seed = 2023

# shuffle data
df_pos = df_pos.sample(frac = 1, replace=False)
df_pos.reset_index(inplace=True)
df_neg = df_neg.sample(frac = 1, replace=False)
df_neg.reset_index(inplace=True)

# sample positive and negative labels
data_pos = df_pos.sample(n=4250, replace=False, random_state=seed)
data_neg = df_neg.sample(n=4250, replace=False, random_state=seed)


# sample train-test split positive labels
test_pos = data_pos.sample(n=500, replace=False, random_state=seed)
train_pos = data_pos[~data_pos.index.isin(test_pos.index)]

# sample train-test split negative labels
test_neg = data_neg.sample(n=500, replace=False, random_state=seed)
train_neg = data_neg[~data_neg.index.isin(test_neg.index)]

# build train and test
testset = pd.concat([test_neg, test_pos])
trainset = pd.concat([train_neg, train_pos])

print()
print(f"len test: {len(testset)}")
print(f"len train: {len(trainset)}")


len test: 1000
len train: 7500


In [73]:
testset.to_csv("../rotten/test.csv", sep="\t", index=False)
trainset.to_csv("../rotten/train.csv", sep="\t", index=False)

In [74]:
def produce_datasets(n, m, out_dir):
     train_set = pd.read_csv("../rotten/train.csv", sep="\t")
     print(len(train_set))

     df_origin = train_set.sample(n=n+m, replace=False, random_state=seed)
     df_n = df_origin.sample(n=n, replace=False, random_state=seed)
     df_seed = df_n.sample(n=m, replace=False, random_state=seed)

     print(f"len origin: {len(df_origin)}")
     print(f"len n_data: {len(df_n)}")
     print(f"len seed: {len(df_seed)}")

     df_origin = trainset.sample(n=n+m, replace=False, random_state=seed)
     df_n = df_origin.sample(n=n, replace=False, random_state=seed)
     df_seed = df_n.sample(n=m, replace=False, random_state=seed)

     print(f"len origin: {len(df_origin)}")
     print(f"len n_data: {len(df_n)}")
     print(f"len seed: {len(df_seed)}")

     df_seed["label_counter"] = [int(not el) for el in df_seed["labels"]]
     df_seed["sentiment_counter"] = ["positive" if x=="negative" else "negative" for x in df_seed["sentiment"]]
     df_seed.rename(columns={"labels": "label_ex",
                             "text": "example",
                             "sentiment": "sentiment_ex"}, inplace=True)

     df_seed["paired_id"] = [i for i in range(len(df_seed))]
     df_seed["counterfactual"] = ["None" for i in range(len(df_seed))]

     df_origin.to_csv(f"../rotten/{out_dir}/origin_data.csv", sep="\t", index=False)
     df_n.to_csv(f"../rotten/{out_dir}/n_data.csv", sep="\t", index=False)
     df_seed.to_csv(f"../rotten/{out_dir}/seed_data.csv", sep="\t", index=False)

## n=5k & m=2.5k (n=2m)

In [75]:
produce_datasets(5000, 2500, "tr2")

7500
len origin: 7500
len n_data: 5000
len seed: 2500
len origin: 7500
len n_data: 5000
len seed: 2500


In [76]:
produce_datasets(2000, 1000, "tr1")

7500
len origin: 3000
len n_data: 2000
len seed: 1000
len origin: 3000
len n_data: 2000
len seed: 1000


The final size of each training set will be m+m+m=2m+m=n+m. The following sets are sampled from train.csv

- origin.csv stores n+m original data points
- m_data.csv
- seed_data.csv is a sample (size m) from n_data

You now need to produce m countefactuals from seed_data!! You will then use n_data.csv and m generated counterfactuals to train your classfier